# AI Engine With PL Example

The AI engine with PL example demonstrates how to use AI engine for computation,
and use PL for data movement. In this example, to run the matrix multiplication 
on AI engine, we use standard matrix multiplication algorithm.The user can change 
the matrix size and the number of cores utilized at compile-time. The expected 
matrix size must be a multiple of 50 (number of cores used) with the minimum and 
maximum value as 100x100 and 500x500 respectively.

Please note that this example is intended to be a proof of concept only. There
can be other ways of implementation, which can leverage more of the AIE resource
and hence can result in better performance figures.

## Introduction

Consider two matrices A and B, the product of the two, i.e. AxB, is a linear
combination of the the columns of A by matrix B. This means that the elements
in a row (i) of A are multiplied with the elements in a column of B (j) and are
summed up to give the corresponding single element in the matrix AxB at i, j.
This means that if A is an n x m matrix and B is an m x p matrix, then the
corresponding product AxB would have dimensions n x p. Note how the number of
columns of A equals the number of rows in B to make the matrix multiplication
possible.

## Implementationi Details


Data slicing and data movement
----------------------------------------------

To compute matrix multiplication on AIE, matrix A is sliced horizontally and
distributed equally among all the core utilized through the AXI-Stream network.
Matrix B is transposed and feed to the first core in the design element by
element. The first core shares the input matrix B with the next core through the
AXI-Stream connection. As the output received from the cores is in column-major
fashion, hence a transpose of the output matrix is expected.

<img src="pics/data_movement.png">

The application uses the PLIO attribute to make external memory-mapped
connections to or from global memory. These connections can be created between
AIE kernel and the logical global memory port of the hardware platform design
via AXI-Multichannel Direct Access IP in the fabric. In this design, the buffer
descriptors are programmed in the AXI-MCDMA IP to initiate AIE to DDR read
and write transactions in the PS program. The burst-length of the memory-mapped
transaction is 64-bit, with required bandwidth as 1000 MB/s and the memory
addressing as physical.


<img src="pics/aie_app_data_movement.png">


### Compilation Flow

<img src="pics/compilation_flow.png">

There are 2 sets of external interfaces for AI Engine configuration
- AI Engine configuration
	Direct call to AI Engine driver APIs
	CDO parsing APIs
- ELF loading
	RemoteProc APIs

The high-level tool, Vitis, can generate outputs in those 2 formats. Also, the
user can manually implement the application using direct calls and compile the
ELF using the low-level compiler.

The generated aie_control.cpp is cross-compiled to run on the target. The
compile application loads the generated ELF to the corresponding tile through AI
Engine remoteproc instance. The AI Engine configuration is done by calling AI
Engine driver APIs directly or pass CDO object through CDO parser library. The
CDO parser is an external component, and the AI Engine software uses the CDO
APIs.



### Run-time Execution

At run-time, Linux application binary calls (UIO based) AI Engine userspace
driver and (tool generated) run time library, libcardano_api.so. AIE userspace
drivers abstract the libmetal and remoteproc APIs to handle runtime
configuration along with ELF loading.


<img src="pics/runtime_execution.png">

The libmetal provides the IO abstraction to the application, which allows the
application to be platform independent, ex standalone and Linux. So all the IO
in the driver is routed through libmetal, and libmetal can handle platform
specific part.

The Linux UIO subsystem allows to run IO from Linux userspace with a small
kernel module, including the MMIO and interrupt handling. The UIO interfaces are
based on the Linux sysfs, and the AI Engine driver stack utilizes this UIO
subsystem through libmetal, to enable platform-independent AI Engine software
stack. So the major part of the AIE specific code resides in the userspace as a
library, which can be reused between other software platforms such as baremetal.

### Build Application Using PetaLinux Tools

By default, the AIE matrix multiplication application is enabled. To enable,
do following:

petalinux-config -c rootfs
```
    [*] user packages --->
        [*] aie-matrix-multiplication
```
To rebuild the project run,

petalinux-build.

The generated FIT image will be in images/linux/image.ub.

Follow PetaLinux boot process to launch the Linux on the target.
After you see the Linux login prompt, you can log in with user "root" and
password "root".

The AIE ELFs are installed in the `/lib/firmware` directory.
We will need to go to `/lib/firmware` to run the application.

### Sample Output
```
root@xilinx-vc-p-a2197-00-reva-x-prc-01-reva-2019_2_siea:~# cd /lib/firmware/aie
root@xilinx-vc-p-a2197-00-reva-x-prc-01-reva-2019_2_siea:~# aie-matrix-multiplication
PLIO MCDMA> reset_dma
metal: info:      Registered shmem provider linux_shm.
metal: info:      Registered shmem provider ion.reserved.
metal: info:      Registered shmem provider ion.ion_system_contig_heap.
metal: info:      Registered shmem provider ion.ion_system_heap.
metal: info:      device xilinx-aiengine in use by driver uio_dmem_genirq
metal: error:     device platform:f70a0000.aie-npi not found
metal: error:     device platform:f70a0000.aie-npi not found
PLIO MCDMA> allocated ddr mem: 0x7f95e28000.
PLIO MCDMA> pa = 0x70100000.
PLIO MCDMA> Successful
```
### Customizing and Rebuilding
Following are prerequisites if the user wants to make any changes in
software,
  * Cardano and cross-compile binaries for any software-related changes.
  * PetaLinux Tools.

As mentioned earlier, the user can change the number of cores utilized by matrix
multiplication. However, since the data memory immediately available to the core
is limited, reducing the number of cores utilized reduces the maximum matrix
size supported by the application. Within the config.h header file NUM_CORES
macro can be altered to change the number of cores utilized.  The maximum cores
available are 50. With the changes made in the application, care must be taken
by the user that the newly generated configuration is supported by the
underlying hardware design.

To rebuild, go to the meta-user demo recipe files directory:
project-spec/meta-user-recipes-apps/aie-matrix-multiplication/files.
set the CARDANO_ROOT and then call AI engine compiler to build:
```
  export CARDANO_ROOT=<Root_To_Installed_Cardano_which_is_under_Vitis>
  source $CARDANO_ROOT/scripts/cardano_env.sh
  aiecompiler --constraints graph_aie_constraints.aiecst \
              --dataflow src/xgemm.cpp --test-iterations=4096 --verbose \
              -full-program=true --use-phy-shim -use-real-noc=true \
              -Xmapper=-c0 -device VC1902 -phydevice xcvc1902 \
              -stacksize=4608 -heapsize=27136 --enable-async-window
```
After the AIE application is customized and compiled, user can run
`clean-aie-work.sh` to clean the Work directory to remove unncessary files,
only leave the AIE kernels and the aie_control.cpp file.

The Linux AIE graph .cpp file is in the `src/` directory. After you build the
AIE application, if you need to build the Linux control application.

You can use the `Makefile.Linux` in the `aie-matrix-multiplication/files/src`
directory to build the Linux AIE graph control application.
You will need to specify CARDANO_ROOT and the Linux sysroot.

To get the Linux sysroot, you can do this with PetaLinux Tools. Go to the PetaLinux
project directory, run the following command:
```
  $ petalinux-build --sdk
  $ petalinux-build --sysroot
```
The sysroot will be generated in `images/linux/sdk/sysroots/aarch64-xilinx-linux/`
Use Makefile.Linux to rebuild the Linux AIE graph application:
```
  $ make -f Makefile.Linux \
    SYSROOT=<plnx_proj>/images/linux/sdk/sysroots/aarch64-xilinx-linux/ \
    CARDANO_ROOT=<cardano_root>
```

The generated Linux binary will be `aie-matrix-multiplication`.

The user can then rebuild PetaLinux.

NOTE:
 * No hardware change is supported in this version of Vivado for this design.

### PDI Generation
Vivado can generate a boot PDI includesd PS, PL and NoC configuration with
"Generate bitstream" icon from Vivado GUI, but it will not includes the AIE
configuration and AIE ELFs.

In order to includes the AIE configuration and ELFs into the boot PDI, user
will need to update the BIF of the boot PDI generated by Vivado.
Vivado generate the boot PDI and the BIF in the hardware project's
"*.runs/impl_1/" directory, e.g.:
 * ps0_me_dma_wrapper.pdi
 * ps0_me_dma_wrapper.pdi.bif

User will need to generate the AIE configuration CDO with cardano first,
and update the bif to includes the AIE CDO and ELFs.

To generate the AIE CDO, user can go to the cardano generated AIE Work/
directory which is generated while compiling the AIE application:
```
  cd Work/ps/cdo/
```
Source cardano settings,
```
  export CARDANO_ROOT=<Installed_Vitis>/cardano/
  source $CARDANO_ROOT/scripts/cardano_env.sh
```
Run the ./generateAIEConfig from this directory. It will generate aie_cdo.bin
file.
And then please add the following configuration partitions to the boot PDI
BIF file generated by Vivado to includes the AIE configuration and ELFs:
```
 partition
  {
   id = 0x10
   type = cdo
   file = <AIE_APP>/Work/ps/cdo/aie_cdo.bin
  }
  partition
  {
   id = 0x12
   core = aie
   file = <AIE_APP>/Work/aie/0_0/Release/0_0
  }
  ...
  partition
  {
   id = 0x12
   core = aie
   file = <AIE_APP>/Work/aie/<C>_<R>/Release/<C>_<R>
  }

```

And then use bootgen to generate the PDI, you can source vitis settings to use
bootgen. E.g.
```
  bootgen -arch versal -image <BIF> -o <PDI>
```

The "files/aie-pdi-gen.sh" gives an example on how to generate a boot PDI to
include AIE configuration and ELFs or a partial PDI which only contains AIE
configuration and ELFs.
The `files/new-aie.bif` gives an example of a boot PDI BIF to contains AIE.



## Run Demo Example


In [1]:
cd /lib/firmware/aie

In [1]:
!aie-matrix-multiplication

## References
 [1] Vivado User Guide - for hardware related design.

 [2] Vitis User Guide - for AIE application.

 [3] Versal TRM - General subsystem overview.
